In [33]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
blocks = 16
batches = 32
max_iters = 50000
learning_rate = 1e-4
training_percent = 0.7

cuda


In [34]:
with open('grammar.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_len = len(chars)

['\n', ' ', '!', '&', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '}', '´', 'Á', 'à', 'á', 'æ', 'è', 'é', 'í', 'ò', 'ó', 'ù', 'ú', 'Ā', 'ā', 'Ă', 'ă', 'ē', 'ĕ', 'ħ', 'Ī', 'ī', 'Ĭ', 'ĭ', 'ō', 'Ŏ', 'ŏ', 'œ', 'Ū', 'ū', 'ŭ', 'ȳ', '̄', '̆', 'Α', 'Κ', 'Λ', 'Ν', 'Ο', 'Τ', '‘', '’', '“', '”', '⪥', '\ufeff']


In [35]:
str2int = { ch:i for i,ch in enumerate(chars) }
int2str = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [ str2int[c] for c in s ]
decode = lambda l: ''.join([ int2str[i] for i in l ])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([125,   0,   0,   0,   0,  39,  71,  68,  57,  74,  56,  58,  57,   1,
         55,  78,   1,  26,  61,  71,  62,  72,   1,  26,  74,  71,  67,  68,
         76,   1,  54,  67,  57,   1,  73,  61,  58,   1,  38,  67,  65,  62,
         67,  58,   1,  27,  62,  72,  73,  71,  62,  55,  74,  73,  58,  57,
          0,  39,  71,  68,  68,  59,  71,  58,  54,  57,  62,  67,  60,   1,
         43,  58,  54,  66,   1,  54,  73,   1,  61,  73,  73,  69,  21,  10,
         10,  76,  76,  76,   9,  69,  60,  57,  69,   9,  67,  58,  73,   1,
          4,  43])


In [36]:
n = int(0.8*len(data))
train = data[:n]
valid = data[n:]

def get_batch(split):
    data = train if split=='train' else valid
    ix = torch.randint(len(data) - blocks, (batches,))
    # print(ix)
    x = torch.stack([data[i:i+blocks] for i in ix])
    y = torch.stack([data[i+1:i+blocks+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
print(x)
print('target:')
print(y)

inputs:
tensor([[ 62,  67,  72,  73,  71,  74,  56,  73,  68,  71,   1,  68,  59,   1,
          78,  68],
        [ 43,  61,  58,  71,  58,   1,  54,  71,  58,   1,  66,  54,  67,  78,
           1,  73],
        [  1,  72,  54,  62,  57,   7,   1,  73,  61,  54,  73,   1, 122,  73,
          61,  58],
        [ 71,  57,   1,   1,   1,   1,   1,   1,   1,  46,  62,  73,  56,  61,
           9,   0],
        [  9,   0,   0,  43,  61,  58,   1,  42,  74,  69,  58,  71,  65,  54,
          73,  62],
        [ 73,  68,   1,  68,  67,  58,   1,  68,  74,  73,   1,  68,  59,   1,
          72,  58],
        [ 54,  72,   1,  54,  67,   1,  62,  66,  69,  68,  71,  73,  54,  67,
          73,   1],
        [ 59,  54,  66,  68,  74,  72,   1,  68,  71,  54,  73,  68,  71,   7,
           1,  76],
        [ 73,  61,   7,   1,  75,  62,  60,  68,  74,  71,   7,   1,  54,  67,
          57,   1],
        [ 43,  61,  58,   1,  72,  58,  56,  68,  67,  57,   1,  62,  72,   1,
          73,  61],
  

In [37]:
x = train[:blocks]
y = train[1:blocks+1]
for t in range(blocks):
    ctx = x[:t+1]
    target = y[t]
    print('When input is', ctx, 'target is', target)

When input is tensor([125]) target is tensor(0)
When input is tensor([125,   0]) target is tensor(0)
When input is tensor([125,   0,   0]) target is tensor(0)
When input is tensor([125,   0,   0,   0]) target is tensor(0)
When input is tensor([125,   0,   0,   0,   0]) target is tensor(39)
When input is tensor([125,   0,   0,   0,   0,  39]) target is tensor(71)
When input is tensor([125,   0,   0,   0,   0,  39,  71]) target is tensor(68)
When input is tensor([125,   0,   0,   0,   0,  39,  71,  68]) target is tensor(57)
When input is tensor([125,   0,   0,   0,   0,  39,  71,  68,  57]) target is tensor(74)
When input is tensor([125,   0,   0,   0,   0,  39,  71,  68,  57,  74]) target is tensor(56)
When input is tensor([125,   0,   0,   0,   0,  39,  71,  68,  57,  74,  56]) target is tensor(58)
When input is tensor([125,   0,   0,   0,   0,  39,  71,  68,  57,  74,  56,  58]) target is tensor(57)
When input is tensor([125,   0,   0,   0,   0,  39,  71,  68,  57,  74,  56,  58,  57]

In [38]:
class Bigram(nn.Module):
    def __init__(self, vocab_len):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_len, vocab_len)
    def forward(self, index, targets=None):
        logits = self.token_embedding(index)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index
model = Bigram(vocab_len)
m = model.to(device)

ctx = torch.zeros((1, 1), dtype=torch.long, device=device)
generated = decode(m.generate(ctx, max_new_tokens=500)[0].tolist())
print(generated)


O}&5nĕkVYăīòœoo’´C]ΛkĀM“-àō0Īĭ0nAnΑŎ4´1ō&5/œYq--´
/cΚ‘èIΑĂ9’‘JvsT
āĪ8ħTrKytMΑpQ6ŏnjIΑ/?ōS[Mœ]qpàĂqălb]ĕō]íō.òxP8ù ùeΚqx-)96RTriĪ.FOœùZBĀE/ŪRĭwăTí5ī⪥.6oăΟΛmh_!}̄*ăj*JΚ̄uōE)[ūΑ´cétĬ&“-̆cĬf1aΟ4s}-.‘ēOáqrpEl8ΟI[zQffY’æhcP
HWUMgΤsctŪW6}wħL0èĭX“ħĀĕkjMĕō..cc⪥ΤōĔ}UàūDx3FPȳCúéÁx-}nUŎ’ēīesrFcéurQ!,ēòΑhzàéħg6N*lŪB&EŎ!﻿‘Ο!‘]cg!àG‘}Aeyē(áVΛ`ŭ_Τg5gf7ă
ŭ9ŎGœ,Ní],“̆8èóRZÁ’QcŎ”}ĀxKóu.àū,s9̆-WΑĭhC
﻿x!⪥_)̄N2´mŭtxE1æ,á7nYĕōNl“ΑL⪥Xù7gO,ħpM3”X_ā0ΛPΟæēt02P’fi:Oă`*éà:;mĭ‘aĬ´QAōFkkŭiò];mUī⪥aΟ̄-[ΟJdĀĕQ Īæòá0baètĭYq3﻿P


In [ ]:
%%time
optim = torch.optim.AdamW(model.parameters(), lr=learning_rate)
for iter in range(max_iters):
    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)
    optim.zero_grad(set_to_none=True)
    loss.backward()
    optim.step()
print(loss.item())

In [30]:
ctx = torch.zeros((1, 1), dtype=torch.long, device=device)
generated = decode(m.generate(ctx, max_new_tokens=500)[0].tolist())
print(generated)


 hol Wisincidiectitus th  wake   f  a lo En thoond  ofof cherd oofe ds ais _I apifory Lora  Wingembyle and tha hedert st ce nirasoie iorlf_llis.” wonusong _tesshe

tholerawigenveque  os  us; by ly, Mreapou ing BUBulde kn the wit;  stute tindjoaylofe  ito
carer iver. vevee who as, veven d titenge II.”  s thet  a   w;
RULild  in, bobe s
ivere  bes nd prrupu me rouin
  t ther-ng qurant e ick “dar IOfre toutht qur an
imorshalppt
ase ay a te.” ase anthispilicurd  e “Expliosere ton, asta se  ront alll
